In [ ]:
!pip install -q -U transformers accelerate bitsandbytes torch fastapi "uvicorn[standard]" pyngrok nest-asyncio sqlalchemy psycopg2-binary pydantic-settings

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from huggingface_hub import login
from google.colab import userdata
import uvicorn
from pyngrok import ngrok
import nest_asyncio

from fastapi import FastAPI, Depends, HTTPException
from sqlalchemy.orm import Session
from sqlalchemy import text
import models
import schemas
import database

In [ ]:
# GİRİŞ YAPMA
try:
    hf_token = userdata.get('HF_TOKEN')
    login(token=hf_token)
    print("Hugging Face girişi Colab Secrets ile başarıyla yapıldı.")

    ngrok_token = userdata.get('NGROK_TOKEN')
    ngrok.set_auth_token(ngrok_token)
    print("ngrok kimlik doğrulaması başarıyla yapıldı.")
except Exception as e:
    print(f"Bir hata oluştu. Lütfen Colab'de 'HF_TOKEN' ve 'NGROK_TOKEN' adında gizli anahtarlar oluşturduğunuzdan emin olun. Hata: {e}")

In [ ]:
# LLM VE PİPELİNE KURULUMU
def initialize_pipeline():
    print("Model ve pipeline başlatılıyor...")
    model_id = "Trendyol/Llama-3-Trendyol-LLM-8b-chat-v2.0"
    bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True)
    hf_pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=2048, temperature=0.3, repetition_penalty=1.15, do_sample=True, top_p=0.9, top_k=50)
    print("Model ve pipeline başarıyla başlatıldı.")
    return hf_pipe

def generate_and_clean(pipe, messages):
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, return_full_text=False, eos_token_id=pipe.tokenizer.eos_token_id)
    return outputs[0]["generated_text"].strip()

# AJANLAR
class Planner:
    def __init__(self, pipe): self.pipe = pipe
    def plan(self, topic):
        system_prompt = (
            "Sen, alanında uzman bir içerik stratejisti ve planlayıcısısın. "
            "Görevin, verilen herhangi bir konu için kapsamlı, detaylı ve mantıksal olarak tutarlı bir blog yazısı planı oluşturmaktır. "
            "Konunun ne olduğundan bağımsız olarak, o konuyu en mantıklı şekilde alt başlıklara ayırmalısın."
            "Çıktın SADECE Markdown formatında başlıklar (#, ##, ###) içermelidir. ASLA açıklama veya paragraf yazma."
            "\n"
            "Planın genel yapısı şu şekilde olmalıdır:"
            "\n"
            "1. Bir '# Giriş' bölümü."
            "\n"
            "2. Konuyu anlamlı bir şekilde bölen, en az 3 ana tema için '##' ile başlayan başlıklar."
            "\n"
            "3. Her ana temanın altına, o temayı detaylandıran en az 2 alt başlık için '###' ile başlayan başlıklar."
            "\n"
            "4. Bir '# Sonuç' bölümü."
        )
        messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": f"Konu: '{topic}' için Markdown formatında detaylı bir plan oluştur."}]
        return generate_and_clean(self.pipe, messages)
class Writer:
    def __init__(self, pipe): self.pipe = pipe
    def write(self, plan):
        system_prompt = (
           "Sen, akıcı ve detaylı bir üsluba sahip profesyonel bir blog yazarısın. "
           "Sana verilen Markdown formatındaki plana harfiyen uyarak, her bir başlık için en az 2-3 paragraftan oluşan, zengin ve bilgilendirici içerik oluştur. "
           "Başlıkları metnin içinde koru. Paragraflar arası geçişlerin yumuşak olmasına özen göster. "
           "Yazında yabancı dilde kelimeler kullanma. Sadece saf, akıcı ve kapsamlı bir Türkçe kullan. "
           "Okuyucuyu sıkmadan konuyu derinlemesine işle."
        )
        messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": f"Aşağıdaki plana göre, her başlığı doldurarak bir blog yazısı yaz:\n\n{plan}"}]
        return generate_and_clean(self.pipe, messages)
class Editor:
    def __init__(self, pipe): self.pipe = pipe
    def edit(self, draft):
        system_prompt = (
            "Sen, titiz ve deneyimli bir baş editörsün. Görevin, sana verilen taslak metni mükemmelleştirmektir. "
            "Metnin ana yapısını, başlıklarını ve uzunluğunu koruyarak cümleleri yeniden yaz, akıcılığı artır ve "
            "varsa anlamsal hataları, imla hatalarını ve yazım yanlışlarını düzelt. "
            "Paragrafları ASLA birleştirme, kısaltma veya özetleme, sadece daha profesyonel ve okunabilir hale getir. "
            "Amacın, metni yayınlanmaya hazır hale getirmektir. Sadece düzenlenmiş nihai metni çıktı olarak ver."
        )
        messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": f"Aşağıdaki taslak metni düzenle ve son halini ver:\n\n{draft}"}]
        return generate_and_clean(self.pipe, messages)

In [ ]:
pipe = initialize_pipeline()
planner = Planner(pipe)
writer = Writer(pipe)
editor = Editor(pipe)

In [ ]:
models.Base.metadata.create_all(bind=database.engine)

In [ ]:
from sqlalchemy import text

try:
    print("Veritabanı bağlantısı kontrol ediliyor ve uyandırılıyor...")
    db_wakeup = database.SessionLocal()
    db_wakeup.execute(text("SELECT 1")) # Veritabanına basit bir "ping" sorgusu gönder
    db_wakeup.close()
    print("✅ Veritabanı bağlantısı başarılı ve aktif.")
except Exception as e:
    print(f"❌ Veritabanına bağlanılamadı. Hata: {e}")
    print("Lütfen Neon.tech projenizin aktif olduğundan ve DATABASE_URL'nin doğru olduğundan emin olun.")

In [ ]:
# FastAPI uygulamasını oluştur
app = FastAPI(title="AI Blog Yazarı API (Veritabanı Destekli)", version="3.0.0")

In [ ]:
# Henüz kullanıcı kayıt sistemimiz olmadığı için, makaleleri bir kullanıcıya bağlamak amacıyla
# veritabanına elle bir test kullanıcısı ekle
db_for_setup = database.SessionLocal()
user_exists = db_for_setup.query(models.User).filter(models.User.id == 1).first()
if not user_exists:
    print("İlk test kullanıcısı (ID=1) oluşturuluyor...")
    test_user = models.User(id=1, username="testuser", email="test@test.com", hashed_password="notarealpassword")
    db_for_setup.add(test_user)
    db_for_setup.commit()
    print("Test kullanıcısı oluşturuldu.")
else:
    print("Test kullanıcısı zaten mevcut.")
db_for_setup.close()

In [ ]:
@app.get("/", summary="API Durum Kontrolü")
def read_root():
    return {"message": "AI Blog Yazarı API'sine hoş geldiniz! Veritabanı bağlantısı aktif."}

# Yeni bir makale oluştur ve ilk planını yap
@app.post("/articles/", response_model=schemas.Article, summary="Yeni Makale Oluştur ve Planla")
def create_article_and_plan(
    title: str,
    db: Session = Depends(database.get_db)
):
    print(f"Yeni makale ve plan isteği alındı: '{title}'")
    plan_text = planner.plan(title)
    print("Plan oluşturuldu.")

    # Makaleyi veritabanına kaydet
    db_article = models.Article(
        title=title,
        plan_content=plan_text,
        status="writing_pending",
        owner_id=1 # tüm makaleleri test kullanıcısına bağla
    )
    db.add(db_article)
    db.commit()
    db.refresh(db_article)
    print(f"Makale (ID: {db_article.id}) veritabanına kaydedildi.")
    return db_article

# Mevcut bir makalenin planını güncelle
@app.put("/articles/{article_id}/plan", response_model=schemas.Article, summary="Makale Planını Güncelle")
def update_article_plan(
    article_id: int,
    request: schemas.PlanUpdateRequest,
    db: Session = Depends(database.get_db)
):
    db_article = db.query(models.Article).filter(models.Article.id == article_id).first()
    if not db_article:
        raise HTTPException(status_code=404, detail="Makale bulunamadı")

    db_article.plan_content = request.new_plan
    db.commit()
    db.refresh(db_article)
    print(f"Makale planı (ID: {article_id}) başarıyla ve düzgün bir şekilde güncellendi.")
    return db_article

# Makaleyi yaz ve düzenle
@app.post("/articles/{article_id}/write", response_model=schemas.Article, summary="Makaleyi Yaz ve Düzenle")
def write_and_edit_article(
    article_id: int,
    db: Session = Depends(database.get_db)
):
    print(f"Yazma isteği alındı (Makale ID: {article_id})")
    db_article = db.query(models.Article).filter(models.Article.id == article_id).first()
    if not db_article:
        raise HTTPException(status_code=404, detail="Makale bulunamadı")

    # Önce yaz ve taslağı kaydet
    draft_text = writer.write(db_article.plan_content)
    print("Taslak metin oluşturuldu.")
    db_article.draft_content = draft_text # TASLAĞI VERİTABANINA YAZ

    # Sonra düzenle ve nihai metni kaydet
    final_text = editor.edit(draft_text)
    print("Nihai metin düzenlendi.")
    db_article.final_content = final_text

    # Durumu güncelle ve kaydet
    db_article.status = "completed"
    db.commit()
    db.refresh(db_article)
    print(f"Makale (ID: {article_id}) güncellendi ve tamamlandı.")
    return db_article

# Veritabanındaki tüm makaleleri listele
@app.get("/articles/", response_model=list[schemas.Article], summary="Tüm Makaleleri Listele")
def read_articles(skip: int = 0, limit: int = 100, db: Session = Depends(database.get_db)):
    articles = db.query(models.Article).order_by(models.Article.id.desc()).offset(skip).limit(limit).all()
    return articles

In [ ]:
# Sunucu başlatma
public_url = ngrok.connect(8000)
print(f"✅ Sunucu genel internete açıldı! Adresiniz: {public_url}")
print("Bu adresi kopyalayıp yerel bilgisayarınızdaki Streamlit uygulamasında kullanın.")

nest_asyncio.apply()
uvicorn.run(app, port=8000)